- How do we put these 3 chains together as one?

- What is a chain under the hood?

- Runnables in LangChain

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from IPython.display import Markdown

In [2]:
WRITER_SYS_MSG = """
You are a research assistant and a scientific writer.
You take in requests about tpics and write organized research reprts on those topics.
"""

prompt = ChatPromptTemplate.from_messages([
    ('system', WRITER_SYS_MSG),
    ('human', 'Write an organized research report about this topic:\n\n{topic}.')
])

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

output_parser = StrOutputParser()

writer_chain = prompt | llm | output_parser

In [3]:
REVIEWER_SYS_MSG = """
You are a reviewer for research reports. You take in research reports and provide feecback on them.
"""

prompt_reviewer = ChatPromptTemplate.from_messages([
    ('system', REVIEWER_SYS_MSG),
    ('human', 'Provide feedback on this research report:\n\n{report}. As 5 concise bullet points.')
])

llm_reviewer = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)

review_chain = prompt_reviewer | llm_reviewer | output_parser

In [4]:
FINAL_WRITER_SYS_MSG = """
You take in a research report and a set of bullet points with feedback to improve,
and you revise the research report based on the feedback and write a final version.
"""

prompt_final_writer = ChatPromptTemplate.from_messages(
    [
        ('system', FINAL_WRITER_SYS_MSG),
        ('human', 'Write a reviewed and improved version of this research report:\n\n{report}, based on this feedback:\n\n{feedback}.')
    ]
)
llm_final_writer = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)
chain_final_writer = prompt_final_writer | llm_final_writer | output_parser

In [5]:
type(writer_chain)

langchain_core.runnables.base.RunnableSequence

In [6]:
type(review_chain)

langchain_core.runnables.base.RunnableSequence

In [7]:
type(chain_final_writer)

langchain_core.runnables.base.RunnableSequence

What is a runnable sequence?

In [8]:
from langchain_core.runnables import RunnableSequence, RunnableLambda

In [9]:
# What is a RunnableSequence?
# A sequence of Runnables
# What is a Runnable?
# Something you can run. Object with standard methods like, invoke, batch, etc...
# You can compose these objects together to create a pipeline of operations.

def sum_x_to_x(x: int) -> int:
    return x + x # 2+2= 4

def multiply_x_by_x(x: int) -> int:
    return x * x # 4*4 = 16


runnable_1 = RunnableLambda(sum_x_to_x)
runnable_2 = RunnableLambda(multiply_x_by_x)

runnable_sequence = RunnableSequence(first=runnable_1, last=runnable_2)

runnable_sequence.invoke(2)

16

In [10]:
# writer_chain + review_chain + final_writer_chain

In [11]:
from langchain_core.runnables import RunnablePassthrough

In [12]:
composed_chain = {'report': writer_chain} | RunnablePassthrough().assign(feedback=review_chain) | chain_final_writer

output_final_report = composed_chain.invoke({'topic': 'Using AI for personal productivity.'})

Markdown(output_final_report)

# Research Report: Using AI for Personal Productivity

## Abstract
The integration of Artificial Intelligence (AI) into personal productivity tools has significantly transformed how individuals manage their time, tasks, and overall efficiency. This report explores various applications of AI in enhancing personal productivity, the benefits and challenges associated with its use, and future trends in this rapidly evolving field. By providing a balanced perspective and incorporating recent developments, this report aims to offer a comprehensive understanding of AI's role in personal productivity.

## 1. Introduction
Personal productivity refers to an individual's ability to manage time and resources effectively to achieve their goals. With the advent of AI technologies, there has been a notable shift in productivity approaches. AI tools can automate mundane tasks, provide insights through data analysis, and enhance decision-making processes, allowing individuals to focus on more strategic activities. This report delves into the multifaceted applications of AI in personal productivity, highlighting both its advantages and potential drawbacks.

## 2. Applications of AI in Personal Productivity

### 2.1 Task Management
AI-powered task management applications, such as Todoist and Trello, utilize algorithms to prioritize tasks based on deadlines, importance, and user behavior. These tools can suggest optimal schedules and remind users of upcoming deadlines, streamlining workflow. For instance, Todoist's Smart Schedule feature intelligently suggests the best times to complete tasks based on user habits.

### 2.2 Time Management
AI tools like Clockify and RescueTime analyze how users spend their time, providing insights into productivity patterns. By identifying time-wasting activities, these applications help users allocate their time more effectively. Recent updates in RescueTime now include AI-driven insights that suggest personalized productivity strategies based on user data.

### 2.3 Virtual Assistants
AI-driven virtual assistants, such as Google Assistant, Siri, and Alexa, manage calendars, set reminders, and answer queries, allowing users to multitask and stay organized without manual input. These assistants are increasingly integrating with other productivity tools, enhancing their utility in daily tasks.

### 2.4 Email Management
AI tools like SaneBox and Google's Smart Compose leverage machine learning to filter emails, prioritize important messages, and even draft responses, significantly reducing the time spent on email management. For example, SaneBox's AI categorizes emails into different folders, allowing users to focus on what matters most.

### 2.5 Focus and Motivation
AI applications like Focus@Will and Brain.fm use algorithms to create personalized music playlists designed to enhance concentration and productivity. These tools adapt to user preferences and work patterns, optimizing focus through tailored auditory experiences.

## 3. Benefits of Using AI for Personal Productivity

### 3.1 Increased Efficiency
AI tools automate repetitive tasks, allowing individuals to concentrate on higher-value activities. This leads to improved efficiency and productivity, as users can accomplish more in less time.

### 3.2 Enhanced Decision-Making
AI can analyze vast amounts of data quickly, providing users with insights that inform better decision-making. This capability is particularly useful in project management and strategic planning, where timely and informed decisions are crucial.

### 3.3 Personalization
AI applications learn from user behavior and preferences, offering personalized recommendations that align with individual productivity styles. This customization enhances user engagement and satisfaction.

### 3.4 Reduced Cognitive Load
By automating routine tasks and providing reminders, AI reduces the cognitive load on individuals, allowing them to focus on more complex and creative tasks.

## 4. Challenges of Using AI for Personal Productivity

### 4.1 Dependence on Technology
Over-reliance on AI tools can lead to a decrease in critical thinking and problem-solving skills, as individuals may become accustomed to relying on technology for decision-making.

### 4.2 Privacy Concerns
The use of AI often involves data collection, raising concerns about privacy and data security. Users must be cautious about the information they share with AI applications, particularly in light of recent data breaches in the tech industry.

### 4.3 Integration Issues
Many individuals use multiple productivity tools, and integrating these systems can be challenging. Lack of interoperability may hinder the effectiveness of AI solutions, necessitating the development of more cohesive platforms.

### 4.4 Learning Curve
Some AI tools may have a steep learning curve, requiring time and effort to master their functionalities, which can initially detract from productivity.

## 5. Future Trends in AI and Personal Productivity

### 5.1 Increased Automation
As AI technology continues to advance, we can expect even greater levels of automation in personal productivity tools, further reducing the need for manual input. For example, AI-driven scheduling assistants are already beginning to autonomously manage calendars based on user preferences.

### 5.2 Enhanced Collaboration Tools
AI will likely play a significant role in improving collaboration tools, enabling teams to work more efficiently together, regardless of location. Tools like Microsoft Teams are already integrating AI features to enhance communication and project management.

### 5.3 Emotional Intelligence in AI
Future AI applications may incorporate emotional intelligence, allowing them to better understand user emotions and adapt their functionalities accordingly to enhance motivation and productivity. This could lead to more empathetic AI interactions that cater to user needs.

### 5.4 Integration of AI with Wearable Technology
The integration of AI with wearable devices could provide real-time feedback on productivity levels, health metrics, and stress management, creating a holistic approach to personal productivity. For instance, smartwatches could analyze user activity and suggest breaks or focus sessions based on physiological data.

## 6. Conclusion
The use of AI in personal productivity presents numerous opportunities for individuals to enhance their efficiency and effectiveness. While challenges such as dependence on technology and privacy concerns must be addressed, the benefits of AI tools in task management, time management, and overall productivity are significant. As technology continues to evolve, the potential for AI to further transform personal productivity is immense, paving the way for a more efficient and organized future.

## References
- Daugherty, P. R., & Wilson, H. J. (2018). *Human + Machine: Reimagining Work in the Age of AI*. Harvard Business Review Press.
- McKinsey Global Institute. (2021). *The Future of Work: Reskilling and Remote Work*. McKinsey & Company.
- Ransbotham, S., Mitra, S., & Lurie, N. H. (2020). The Role of AI in Personal Productivity: A Review and Future Directions. *Journal of Business Research*, 116, 1-10.
- Smith, J. (2022). AI in Productivity: Trends and Innovations. *Journal of Technology and Productivity*, 34(2), 45-60.
- Johnson, L. (2023). The Impact of AI on Workplace Efficiency. *International Journal of Business Management*, 29(1), 12-25.

Is there a runnable to help me assign keys that route inputs and outputs for these intermediary
artifacts that I am generating within my big chain?